In [ ]:
%load_ext autoreload
%autoreload 2
from lib import load, timing

YEAR = 2024
DAY = 6
TESTDATA = [
    None,
    '....#.....\n.........#\n..........\n..#.......\n.......#..\n..........\n.#..^.....\n........#.\n#.........\n......#...\n'
][0]
TEST = TESTDATA is None

In [ ]:
from lib import CharGrid

@timing
def prepare_data():
    data = load(YEAR, DAY, split_lines=True, test=TESTDATA if TEST else None)
    grid = CharGrid(data['split'])
        
    guard = grid.find('^')
    grid[guard] = '.'

    return grid, guard

In [ ]:
# Level 1: Track position of guard walking through the grid, counting their steps

import numpy as np

def visit(grid, pos, newvalue = None):
    if newvalue is not None:
        grid[pos[1]][pos[0]] = newvalue
    return grid[pos[1]][pos[0]]

def trace_guard(grid, start):
    history = set()
    grid_size = np.array(grid.shape)
    direction = 0 # 0 - north, 1 - east, 2 - south, 3 - west, q.v. grid.neighbours()
    position = start
    while grid.is_inside(position):
        state = (position, direction)
        if state in history:
            return None
        history.add(state)
        nextp = grid.neighbours(position)[direction]
        if grid.is_inside(nextp) and grid[nextp] == '#':
            direction = (direction + 1) % 4
        else:
            position = nextp
    return set([pos for pos, dir in history])

@timing
def level1(grid, initial):
    path = trace_guard(grid, initial)
    return len(path)


grid, initial = prepare_data()
print(level1(grid, initial))

In [ ]:
# Level 2: Find positions for obstacles that force the guard on a closed loop
@timing
def level2(grid, initial):
    loopers = []
    for pos in trace_guard(grid, initial):
        if pos != initial:
            grid[pos] = '#'
            if trace_guard(grid, initial) is None:
                loopers.append(pos)
            grid[pos] = '.'
    return len(loopers)

grid, initial = prepare_data()
print(level2(grid, initial))